# Introduction to Generative Models

本章主要介绍以下内容：
- generative models, 以及和discrimitive models 之间的区别
- 使用Naïve Bayes 构建第一个generative model，当scale 增大是这个模型会fail

## 1. Introduction to generative modeling

### 1.1 什么事generative model?

Generative model 的官方定义如下：

A generative model describes how a dataset is generated, in terms of a probabilistic model. By sampling from this model, we are able to generate new data.

generative model 的流程如下：
- 首先我们需要有一个包含很多希望生成对象的training dataset，例如下图所示，我们可以有许多马的图片。每个training example可以称作一个observation。每个observation 可以有很多features，对于图像来说，每个pixel 可以看作一个feature.

- 我们的目标是: generate new sets of features that look as if they have been created using the same rules as the original data.

build a model that mimics this distribution as closely as possible and then sample from it to generate new, distinct observations that look as if they could have been included in the original training set.

<img src="figures/generative_modeling.png" alt="drawing" width="600"/>

A generative model must also be probabilistic rather than deterministic. The model must include a stochastic (random) element that influences the individual samples generated by the model.

下图展示了生成模型从2014 - 2018年的演进。

<img src="figures/gan_progress.png" alt="drawing" width="600"/>



### 1.2 Generative modeling 对比 Discriminative modeling

大部分分类问题使用的是discriminative models。例如，给一幅画，我们来预测这幅画是否是梵高的作品。

<img src="figures/discrimitive_modeling.png" alt="drawing" width="600"/>

#### discrimitive modeling
通常，discrimitive model 都需要标签，所以一般都是supervised learning. 

Discriminative modeling estimates p(y|x): the probability of a label y（是否是梵高的作品） given observation x（一幅画）.

#### generative modeling

可以用在unlabeled datasets和labeled datasets（对每个class 生成一个generative model）.

Generative modeling estimates p(x): the probability of observing observation x.
If the dataset is labeled, we can also build a generative model that estimates the distribution
p(x|y). 给一个类别（例如马），学习一个分布，然后通过sampling 生成一个sample（例如一幅马的图片）。

与discrimitive model 相比，Generative models 更难评估，因为生成的结果好坏通常是一个主管的评估，而不像discrimitive model 那样可以用一个指标来衡量。

discrimitive model 比generative model 更有商业应用场景。

### 1.3 我们为什么需要generative modeling?

1. 从纯学术和研究的角度，我们应该探讨数据是怎样生成的。more complete understanding of how the data was generated in the first place。这是一个更为困难的问题。原因是生成一个图像，可能的空间太大，而真正符合我们要求的图片又很少（例如，256*256 的图片，是一只猫）。

2. Generative model可以驱动其他领域的研究，例如强化学习。if the agent were able to simulate its environment through a generative model, it wouldn’t need to test out the strategy in the computer simulation

3. Generative model 和人体智能体更接近。

Current neuroscientific theory suggests that our perception of reality is not a highly complex
discriminative model operating on our sensory input to produce predictions of what
we are experiencing, but is instead a generative model that is trained from birth to
produce simulations of our surroundings that accurately match the future.

## 2. Probabilistic generative models

### 2.1 Generative Modeling Framework

1. 我们又一个数据集，observations X
2. 我们假设这个数据集是由一个分布生成的，记为$p_{data}$
3. Generative model $p_{model}$ 试图还原$p_{data}$（可以说$p_{model}$ 是对$p_{data}$的一个估计）
4. 我们基于$p_{model}$做sampling，如果$p_{model}$ 足够逼近$p_{data}$，那我们就可以生成十分逼真的图片

注意，这里$p_{model}$ sampling 的内容不能只是reproduce 原始数据集的数据。

#### 例子

我们有如下data：

<img src="figures/example_data.png" alt="drawing" width="600"/>

我们可能猜想，数据是基于中间有色方框区域随机采样生成的。这就是我们的$p_{model}$。 基于这个$p_{model}$， 我们可以随机选出更多的点。

<img src="figures/example_p_model.png" alt="drawing" width="600"/>

其实真正的数据是基于一个世界地图，所有人口超过100w 的城市的采样，如下图所示。因此，我们的$p_{model}$和真实的$p_{data}$相差很远（oversimplified），在这个场景下，我们生成的数据（点）有时候放在地图上就会感到很奇怪，这就是不好的生成。

- A 点，在海上。
- B 点，和已有点过于接近，也属于不好的生成
- C 点，好的生成。

<img src="figures/example_p_data.png" alt="drawing" width="600"/>

从这个例子中，我们可以看出生成模型的复杂性。如果要生成衣服图像，通常是上万维度的数据，更容易输出“不好”的结果，所以如何控制generative model 生成我们想要的内容是一个十分有挑战性的工作。

### 2.2 Probabilistic Generative Models

我们首先，不基于deep learning，而是基于probability 来构建一个generative model.

我们需要定义4 key terms: 
1. sample space
2. density function
3. parametric modeling
4. maximum likelihood estimation

#### 1. sample space
The sample space is the complete set of all values an observation x can take.

例如在上面的例子中，二维平面中所有的点就是我们的sample space。

#### 2. Probability Density Function
$p(x)$: 将采样空间中的每个点映射到一个0-1之间的值。采样空间中所有的点的density function相加为1.

在上面的例子中，在box 外是0，在box 内是固定值（取决于点的精度）。

通常，这样的density 函数有很多，通常，为了找到最合适的密度函数，我们使用parametric modeling 方法。

#### 3. Parametric Modeling
$p_{\theta}(x)$: is a family of density functions that can be described using a finite number of parameters, $\theta$.

在上面的例子中，不同的box 可以看作是不同参数控制的model.所以这个model 有四个参数，分别是左上角和右下角的坐标。

#### 4. Likelihood
$\mathscr{L}(\theta | x) = p_{\theta}(x)$ measures the plausibility of θ, given some observed point x.

如果我们有很多独立的观察样本X，我们只需要将每个样本的饿liklihood 相乘即可。
$\mathscr{L}(\theta | X) = \prod_{x \in X}p_{\theta}(x)$

乘机操作起来比价复杂，所以我们可以使用log-likelihood 代替。

$\mathscr{l}(\theta | X) = \sum_{x \in X}log p_{\theta}(x)$

#### 最大似然估计 maximum likelihood estimation
这样，我们可以找到最优的参数$\hat{\theta}$ 来maximize the likelihood of observating the dataset X.

$$\hat{\theta} = argmax_{\theta}\mathscr{L}(\theta | X)$$


## 3. The challenges of generative modeling

## References

[1] A Style-Based Generator Architecture for Generative Adversarial Networks

[2] Language Models Are Unsupervised Multitask Learners

[3] The Malicious Use of Artificial Intelligence: Forecasting, Prevention, and Mitigation